In [1]:
import tensorflow as tf

ModuleNotFoundError: No module named 'tensorflow'

In [2]:
print(tf.__version__)

NameError: name 'tf' is not defined

In [3]:
import keras

ModuleNotFoundError: No module named 'keras'

In [4]:
print(keras.__version__)

NameError: name 'keras' is not defined

## Import the dataset

In [14]:
from sklearn.datasets import load_files       
from keras.utils import np_utils
import numpy as np
from glob import glob

# define function to load train, test, and validation datasets
def load_dataset(path):
    data = load_files(path)
    dog_files = np.array(data['filenames'])
    dog_targets = np_utils.to_categorical(np.array(data['target']), 133)
    return dog_files, dog_targets

# load train, test, and validation datasets
# train_targets are one-hot encodingss
train_files, train_targets = load_dataset('dogImages/train')
valid_files, valid_targets = load_dataset('dogImages/valid')
test_files, test_targets = load_dataset('dogImages/test')

# print statistics about the dataset
print('There are %d total dog categories.' % len(dog_names))
print('There are %s total dog images.\n' % len(np.hstack([train_files, valid_files, test_files])))
print('There are %d training dog images.' % len(train_files))
print('There are %d validation dog images.' % len(valid_files))
print('There are %d test dog images.'% len(test_files)) 

FileNotFoundError: [Errno 2] No such file or directory: 'dogImages/valid'

## Preprocessing functions

In [ ]:
from keras.preprocessing import image                  
from tqdm import tqdm

def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

## Prediction

### Load the labels

In [10]:
import numpy

dog_names = numpy.load('./labels/dog_names.npy')
print(len(dog_names))

133


### Prediction function

In [12]:
from keras.applications.resnet50 import preprocess_input, decode_predictions

# Input: Imagepath
# Output: string of dogname
def dog_predictor(model, img_path):
    img = preprocess_input(path_to_tensor(img_path))
    prediction = model(img_path)
    return dog_names[np.argmax(ResNet50_model.predict(img))]

## Developing the model

In [ ]:
from PIL import ImageFile                            
ImageFile.LOAD_TRUNCATED_IMAGES = True                 

# pre-process the data for Keras
train_tensors = paths_to_tensor(train_files[:1000]).astype('float32')/255
valid_tensors = paths_to_tensor(valid_files).astype('float32')/255
test_tensors = paths_to_tensor(test_files).astype('float32')/255

print("The shape of input: ", train_tensors[0].shape)

### Define the model

### Compile the model

In [ ]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

### Train the model

In [ ]:
from keras.callbacks import ModelCheckpoint  

epochs=5
checkpointer = ModelCheckpoint(filepath='keras_models/weights.best_single_model.hdf5', 
                               verbose=1, save_best_only=True)

model.fit(train_tensors, train_targets, 
          validation_data=(valid_tensors, valid_targets),
          epochs=epochs, batch_size=20, callbacks=[checkpointer], verbose=1)

### Test accuracy of the model

In [ ]:
model.load_weights('saved_models/weights.best_single_model.hdf5')


# get index of predicted dog breed for each image in test set
dog_breed_predictions = [ np.argmax(model.predict(np.expand_dims(tensor, axis=0))) for tensor in test_tensors ]

# report test accuracy
test_accuracy = 100*np.sum(np.array(dog_breed_predictions)==np.argmax(test_targets, axis=1))/len(dog_breed_predictions)